# Simple example

## Import the BipartitePandas package

Make sure to install it using `pip install bipartitepandas`.

In [1]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

For this notebook, we simulate data (we set parameters to make data cleaning interesting).

In [2]:
df = bpd.SimBipartite(bpd.sim_params({'firm_size': 10, 'p_move': 0.05})).simulate()
display(df)

,i,j,y,t,l,k,alpha,psi
0,0,254,-0.644031,0,3,2,0.430727,-0.604585
1,0,254,-0.976315,1,3,2,0.430727,-0.604585
2,0,254,-0.186705,2,3,2,0.430727,-0.604585
3,0,254,-1.092270,3,3,2,0.430727,-0.604585
4,0,254,-0.716801,4,3,2,0.430727,-0.604585
...,...,...,...,...,...,...,...,...
49995,9999,564,-0.636694,0,4,5,0.967422,0.114185
49996,9999,564,1.264165,1,4,5,0.967422,0.114185
49997,9999,584,-0.051822,2,4,5,0.967422,0.114185
49998,9999,584,2.635244,3,4,5,0.967422,0.114185


## Columns

BipartitePandas includes seven pre-defined general columns:

#### Required
- `i`: worker id (any type)
- `j`: firm id (any type)
- `y`: income (float or int)

#### Optional
- `t`: time (int)
- `g`: firm type (any type)
- `w`: weight (float or int)
- `m`: move indicator (int)

## Constructing DataFrames

How do we construct a dataframe? Just use the required columns (plus any optional columns you want to include)!

In [3]:
bdf = bpd.BipartiteDataFrame(i=df['i'], j=df['j'], y=df['y'], t=df['t'])
display(bdf)

,i,j,y,t
0,0,254,-0.644031,0
1,0,254,-0.976315,1
2,0,254,-0.186705,2
3,0,254,-1.092270,3
4,0,254,-0.716801,4
...,...,...,...,...
49995,9999,564,-0.636694,0
49996,9999,564,1.264165,1
49997,9999,584,-0.051822,2
49998,9999,584,2.635244,3


## Now that we have our dataframe, let's check out some summary statistics

In [4]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 10000
number of firms: 1001
number of observations: 50000
mean wage: -0.011877194839660522
median wage: -0.01175137487332556
min wage: -5.303053932829651
max wage: 5.960757777562984
var(wage): 2.6747830219232895
no NaN values: False
no duplicates: False
i-t (worker-year) observations unique (None if t column(s) not included): False
no returns (None if not yet computed): None
contiguous 'i' ids (None if not included): False
contiguous 'j' ids (None if not included): False
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): None


## Let's clean our data - and make sure the result is leave-one-observation-out connected

<div class="alert alert-info">

Hint

Want details on all cleaning parameters? Run `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then run `bpd.clean_params().describe(key)`.

</div>

In [5]:
bdf = bdf.clean(bpd.clean_params({'connectedness': 'leave_out_observation'}))
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how='leave_out_observation')
making 'i' ids contiguous
making 'j' ids contiguous
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-0.644031,0,0
1,0,0,-0.976315,1,0
2,0,0,-0.186705,2,0
3,0,0,-1.092270,3,0
4,0,0,-0.716801,4,0
...,...,...,...,...,...
45009,9043,856,-0.636694,0,0
45010,9043,856,1.264165,1,1
45011,9043,12,-0.051822,2,1
45012,9043,12,2.635244,3,0


We can check how the summary statistics changed:

In [6]:
bdf.summary()

format: 'BipartiteLong'
number of workers: 9044
number of firms: 890
number of observations: 45014
mean wage: -0.012294051999150506
median wage: -0.009017844914140066
min wage: -5.303053932829651
max wage: 5.960757777562984
var(wage): 2.641798586664899
no NaN values: True
no duplicates: True
i-t (worker-year) observations unique (None if t column(s) not included): True
no returns (None if not yet computed): False
contiguous 'i' ids (None if not included): True
contiguous 'j' ids (None if not included): True
contiguous 'g' ids (None if not included): None
connectedness (None if ignoring connectedness): 'leave_out_observation'
